In [1]:
# First cell - setup imports
import sys
from pathlib import Path

# Import the package
from topolansatz.topology import TopologyHandler
from topolansatz.circuits import CircuitBuilder
from topolansatz.evaluator import CircuitEvaluator
from topolansatz.stitching import CircuitStitcher
from topolansatz.main import TopolAnsatz

In [2]:
# Test cell with debug prints
coupling_map = [
    (0,1), (1,2), (2,3), (3,4),
    (0,5), (1,6), (2,7), (3,8), (4,9),
    (5,6), (6,7), (7,8), (8,9)
]

print("Creating TopolAnsatz instance...")
topol = TopolAnsatz(coupling_map)

print("\n1. Testing Topology...")
topology = TopologyHandler(coupling_map)
subgraphs = topology.get_connected_subgraphs(4)
print(f"Found {len(subgraphs)} 4-qubit subgraphs")

print("\n2. Testing Circuit Building...")
norm_graph, index_map = topology.normalize_indices(subgraphs[0])
print("Normalized graph nodes:", norm_graph.nodes())
print("Index mapping:", index_map)
builder = CircuitBuilder(norm_graph, "test_circuit")
circuit = builder.create_circuit(depth=2)
display(circuit.draw())

print("\n3. Testing Evaluation...")
evaluator = CircuitEvaluator()
metrics = evaluator.get_circuit_metrics(circuit)
print("Circuit metrics:", metrics)

print("\n4. Testing Full Ansatz Generation...")
# Try with smaller number of qubits first
ansatz = topol.generate_ansatz(n_qubits=6, depth=2)  # Start with 4 qubits
print("Generated ansatz with", ansatz.num_qubits, "qubits")
display(ansatz.draw())

print("\n5. Testing Final Evaluation...")
final_metrics = topol.evaluate_ansatz(ansatz)
print("Final ansatz metrics:", final_metrics)

Creating TopolAnsatz instance...

1. Testing Topology...
Found 32 4-qubit subgraphs

2. Testing Circuit Building...
Normalized graph nodes: [0, 1, 2, 3]
Index mapping: {0: 0, 1: 1, 2: 2, 3: 3}


┌───┐                ┌─────────────────────────┐     »
q_0: ───────────┤ T ├─────────────■──┤ Ry(θ_test_circuit_ry_2) ├─────»
     ┌──────────┴───┴──────────┐┌─┴─┐└─────────────────────────┘┌───┐»
q_1: ┤ Rx(θ_test_circuit_rx_0) ├┤ X ├─────────────■─────────────┤ T ├»
     ├─────────────────────────┤└───┘           ┌─┴─┐           └───┘»
q_2: ┤ Rx(θ_test_circuit_rx_1) ├────────────────┤ X ├─────────────■──»
     └──────────┬───┬──────────┘                └───┘           ┌─┴─┐»
q_3: ───────────┤ S ├───────────────────────────────────────────┤ X ├»
                └───┘                                           └───┘»
«                                      
«q_0: ─────────────■───────────────────
«                ┌─┴─┐                 
«q_1: ───────────┤ X ├────────────X────
«                ├───┤            │    
«q_2: ───────────┤ T ├────────────X──X─
«     ┌──────────┴───┴──────────┐    │ 
«q_3: ┤ Rz(θ_test_circuit_rz_3) ├────X─
«     └─────────────────────────┘


3. Testing Evaluation...
Circuit metrics: {'expressivity': np.float64(0.4878287410796692), 'entanglement': np.float64(0.6803660945976681), 'depth': 7, 'n_parameters': 4, 'n_cnot': 4}

4. Testing Full Ansatz Generation...
Generated ansatz with 6 qubits


»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
q_4: »
     »
q_5: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                             ├
«     │                                                                                              │
«q_1: ┤1                                                                                             ├
«     │                                                                                              │
«q_2: ┤2                                                                                             ├
«     │  circuit-3163(θ_sub_0_rx_2,θ_sub_0_rx_4,θ_sub_0_ry_0,θ_sub_0_ry_1,θ_sub_0_ry_5,θ_sub_0_rz_3) │
«q_3: ┤3                                                                                             ├
«     │                                                                                              │
«q_4: ┤4                                                                                             ├
«     │                                                                                              │
«q_5: ┤5                                                                                             ├
«     └──────────────────────────────────────────────────────────────────────────────────────────────┘


5. Testing Final Evaluation...
Final ansatz metrics: {'expressivity': np.float64(0.4368846912434291), 'entanglement': np.float64(0.7496916175958578), 'depth': 1, 'n_parameters': 6, 'n_cnot': 0}


In [5]:
# Test code for partitioned ansatz
coupling_map = [
    (0,1), (1,2), (2,3), (3,4),
    (0,5), (1,6), (2,7), (3,8), (4,9),
    (5,6), (6,7), (7,8), (8,9)
]

# Create TopolAnsatz instance
topol = TopolAnsatz(coupling_map)

# Test different partitioning schemes
partitioning_schemes = [
    [4, 4, 2],  # Scheme 1: two 4-qubit and one 2-qubit circuits
    [4, 3, 3],  # Scheme 2: one 4-qubit and two 3-qubit circuits
    [3, 3, 2, 2]  # Scheme 3: two 3-qubit and two 2-qubit circuits
]

for i, partition_sizes in enumerate(partitioning_schemes):
    print(f"\nTesting partitioning scheme {i+1}: {partition_sizes}")
    try:
        # Generate partitioned ansatz
        ansatz = topol.generate_partitioned_ansatz(
            total_qubits=10,
            partition_sizes=partition_sizes,
            depth=2
        )
        
        # Display circuit
        print(f"Successfully generated ansatz with {ansatz.num_qubits} qubits")
        print("Circuit structure:")
        display(ansatz.draw())
        
        # Evaluate metrics
        metrics = topol.evaluate_ansatz(ansatz)
        print("Circuit metrics:", metrics)
        
        # Visualize subcircuit structure
        print("\nSubcircuit connectivity:")
        plt.figure(figsize=(8, 6))
        nx.draw(topol.graph, with_labels=True, node_color='lightblue', 
                node_size=500, font_size=16)
        plt.title(f"Partitioning: {partition_sizes}")
        plt.show()
        
    except ValueError as e:
        print(f"Failed to generate ansatz: {e}")


Testing partitioning scheme 1: [4, 4, 2]
